<a href="https://colab.research.google.com/github/toot09/CF_WINGS_POS/blob/master/POS_CF_ONEPERSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Source : https://data-science-hi.tistory.com/73

In [8]:
import numpy as np
import pandas as pd
import math
import operator

In [26]:
# 상품 마스터
items=[]
with open('ITEM_2114.csv','r',encoding='UTF-8') as f:
  for line in f.readlines():
    items.append(line.strip().split(","))
all_item = [i[0] for i in items[1:]]
print(len(all_item))
print(all_item[:2])

537
['고메플레인스콘(1)', '손수동그랑땡1KG(다향)']


In [68]:
# 매출 마스터
sales=[]
with open('POS_2114.csv','r',encoding='UTF-8') as f:
  for line in f.readlines():
    sales.append(line.strip().split(","))
print(sales[:2])

[['\ufeffADDR', 'ITEM', 'SALE'], ['강원도 강릉시', '고메그릴피자불고기360G/냉동(씨제이제일제당)', '6']]


In [122]:
# 1인 가구 비율
tmp=[]
with open('RT_ONE.csv','r',encoding='UTF-8') as f:
  for line in f.readlines():
    tmp.append(line.strip().split(","))
rtOne={}
for i in tmp[1:]:
  rtOne[i[0]] = float(i[1])
print(rtOne)

{'고양시 덕양구': 0.25117018054214, '고양시 일산동구': 0.26290805455411, '고양시 일산서구': 0.18866716363705, '광주광역시 광산구': 0.27208691125091, '광주광역시 남구': 0.2835011082379, '광주광역시 동구': 0.38482906450272, '광주광역시 북구': 0.31562951522096, '광주광역시 서구': 0.32198044780826, '김제시 고창군': 0.34874759152215, '김제시 무주군': 0.34786803009839, '김제시 부안군': 0.33770197486535, '김제시 순창군': 0.32988089073019, '김제시 완주군': 0.3189087977531, '김제시 임실군': 0.37492764409162, '김제시 장수군': 0.33464945043218, '김제시 진안군': 0.34933078393881, '대구광역시 남구': 0.41259015821312, '대구광역시 달서구': 0.27962935351745, '대구광역시 달성군': 0.23703560174184, '대구광역시 동구': 0.29739989562407, '대구광역시 북구': 0.2910230896221, '대구광역시 서구': 0.12146753097694, '대구광역시 수성구': 0.23792925218092, '대구광역시 중구': 0.24250052443885, '대전광역시 대덕구': 0.31045729242322, '대전광역시 동구': 0.37501949317738, '대전광역시 서구': 0.32746638693604, '대전광역시 유성구': 0.34014655633773, '대전광역시 중구': 0.30515306227908, '부산광역시 강서구': 0.23983178379269, '부산광역시 금정구': 0.33715389318607, '부산광역시 기장군': 0.26529019101785, '부산광역시 남구': 0.2969270346942, '부산광역시 동구': 0

In [40]:
# 지역별, 상품별 판매수량 Dict
def sales_dictionary(ratings):
  s_dict={}
  for i in sales[1:]:
    if i[0] not in s_dict:
      s_dict[i[0]] = {i[1]:float(i[2])}
    else:
      s_dict[i[0]].setdefault(i[1],float(i[2]))
  #{ ADDR : { ITEM : SALE }
  return s_dict

In [118]:
# 코사인 유사도
def cosine_similarity(A,B):
  dot_p = np.dot(A,B)
  A_norms = math.sqrt(sum([i**2 for i in A]))
  B_norms = math.sqrt(sum([i**2 for i in B]))
  AB_norms = A_norms*B_norms
  return dot_p/AB_norms

In [69]:
# 지역별 판매현황
slDf = pd.DataFrame(np.array(sales[1:]),columns=sales[0])
# 지역별, 상품별 판매수량 { 지역 : {상품 : 판매수량} }
s_dict = sales_dictionary(slDf)

In [129]:
# 사용자기반 필터링 (User Based Filtering - UBF)
def user_based_filtering(sale_dict, addr, k=3):
  # 지역의 매출 상품 집합(Set)
  addr_s = set(sale_dict[addr].keys())
  similar_score={}
  
  for other_addr in sale_dict.keys():
    # addr과 other_addr의 공통 매출 상품 교집합(intersection)
    both_s = addr_s.intersection(set(sale_dict[other_addr].keys()))
    # 공통된 매출상품이 1개 이상인 경우만 고려
    if (len(both_s)>=1) and (other_addr in rtOne.keys()):
      # 매출수량과 1인가구 비율 차이 가중값으로 코사인 유사도 도출
      addr_s_sale = [sale_dict[addr][i] for i in both_s]
      addr_other_sale = [sale_dict[other_addr][i] for i in both_s]
      similar_score[other_addr] = cosine_similarity(addr_s_sale, addr_other_sale) - abs(rtOne[addr]-rtOne[other_addr])/10000
  
  #유사도 높은 지역순으로 sort
  neighborhood = sorted(similar_score.items(), key=operator.itemgetter(1), reverse=True)
  print("■■■ 지역의 유사도 현황 ■■■")
  print(neighborhood[:50])

  # 추천을 위한 neighborhood 리스트
  addr_for_recommendation = [i[0] for i in neighborhood]
  # addr에 판매기록이 없는 상품리스트
  no_sale_i = set(all_item) - addr_s

  # 추천결과
  recommendation_of_items = {}
  for item in no_sale_i:
    r=[]
    s=[]
    for addr in addr_for_recommendation:
      if item in sale_dict[addr].keys():
        # 1인가구로 계산된 유사도로 예상 매출수량 반영
        r.append(sale_dict[addr][item]*similar_score[addr])
        s.append(similar_score[addr])
      else:
        continue
    # 판매상품기록이 없는 상품이 다른 지역에도 판매되지 않은경우
    if (len(r)==0):
      continue
    # k보다 작거나 같은 판매기록이 있는경우 사용자끼리의 가중평균을 구함
    elif len(r)<=k:
      recommendation_of_items[item] = sum(r)/sum(s)
    # K보다 많은 판매기록이 있는경우 정확도를 위해 상위 k번째의 지역에 대해서만 평균(가중평균)을 둠
    else:
      recommendation_of_items[item] = sum(r[:k])/sum(s[:k])
    
  return sorted(recommendation_of_items.items(), key=operator.itemgetter(1), reverse=True)[:1000]

In [134]:
# 사용자 기반 필터링
recommendation_1000 = user_based_filtering(s_dict, '대구광역시 서구')
pd.DataFrame(np.array(recommendation_1000),columns=["상품명","예상판매수량"])

■■■ 지역의 유사도 현황 ■■■
[('대구광역시 서구', 1.0), ('부산광역시 부산진구', 0.8808604785775682), ('서울특별시 관악구', 0.8142540320721011), ('대구광역시 북구', 0.8055489742043255), ('대구광역시 남구', 0.7855459066989927), ('서울특별시 강서구', 0.7641749102319046), ('대구광역시 달서구', 0.7641044941213585), ('서울특별시 마포구', 0.7272756365311874), ('대전광역시 중구', 0.7245121019003871), ('부산광역시 동구', 0.719815943727491), ('부산광역시 북구', 0.709396524294562), ('부산광역시 연제구', 0.705422686345951), ('제주특별자치도 서귀포시', 0.7020718583097929), ('부산광역시 금정구', 0.6895737036190462), ('대전광역시 서구', 0.688855122136212), ('서울특별시 광진구', 0.6817024652834135), ('서울특별시 용산구', 0.6784896031497708), ('대구광역시 수성구', 0.6744830055551307), ('광주광역시 동구', 0.660580584493581), ('서울특별시 금천구', 0.6570587512716688), ('대구광역시 동구', 0.6562745610987136), ('광주광역시 남구', 0.6561584052457454), ('서울특별시 강동구', 0.6559286292506102), ('대전광역시 동구', 0.653169703856293), ('대전광역시 유성구', 0.6462058618819214), ('서울특별시 성북구', 0.6439280301278013), ('광주광역시 서구', 0.6423364567702614), ('부산광역시 수영구', 0.6399452304250719), ('광주광역시 광산구', 0.6375354563200

,상품명,예상판매수량
0,대림선안심치킨텐더스틱540G(사조),26.36051714833558
1,청정원숯불떡갈비1KG(대상),18.0
2,쉐프원해물볶음밥300G(대상),15.789943240217553
3,쉐프원새우볶음밥300G(대상),14.594745300257399
4,쉐프원햄야채볶음밥300G(대상),14.027990477906338
...,...,...
336,안동식찜닭400G(하림),1.0
337,신비비고언양식바싹불고기460G+92G_HP/냉동(씨제이제일제당),1.0
338,팝콘치킨1KG(한성기업),1.0
339,한성동그랑땡1KG(한성기업),1.0


In [132]:
# 아이템 기반 필터링 (Item Based Filtering - IBF)
def item_based_filtering(sale_dict, addr, k=3):
  # 임시 (addr 별 매출이 있는 상품 list)
  #tmp = [list(sale_dict[i].keys()) for i in list(sale_dict.keys())]
  # 매출이 발생하지 않은 상품들 체크를 위한 list(데이터는 모든 매출이 발생한 상품)
  #no_sale_i = list(j for i in tmp for j in i)
  
  # 매출이 발생하지 않은 상품들 (ujone)
  no_sale_i = set(all_item) - sale_dict[addr].keys()
  # addr에 매출 발생된 상품
  sale_i = [i for i in sale_dict[addr].keys()]

  # 매출 발생되지 않은 상품에 대한 dict
  c_s = {}

  # j:매출발생된 상품 / i:매출발생하지 않은 상품(알아볼 상품)
  for i in no_sale_i:
    c_s[i] = {}
    for j in sale_i:
      p_i = []
      p_j = []
      for p in list(sale_dict.keys()):
        if j in sale_dict[p].keys() and i in sale_dict[p].keys() and p in rtOne.keys():
          p_i.append(sale_dict[p][i] - abs(rtOne[addr]-rtOne[p])/10000)
          p_j.append(sale_dict[p][j] - abs(rtOne[addr]-rtOne[p])/10000)
      if len(p_i)>0:
        c = cosine_similarity(p_i, p_j)
        c_s[i].setdefault(j,c)
        #{ 매출발생하지 않은 상품 : {매출발생된 상품 : 코사인유사도} }
  
  print("■■■ 상품별 유사도 현황 ■■■")
  for i in list(c_s.keys())[:2]:
    tmp = sorted(c_s[i].items(), key=operator.itemgetter(1), reverse=True)
    print("예상 상품 : ",i," 매출발생 상품 : ",tmp[0]," 유사도 : ",tmp[1])
  
  # 결과
  result = {}

  # 매출 발생하지 않은 상품에 대한 코사인 유사도 계산
  for i in list(c_s.keys()):
    sum_sim_rating=[]
    sum_sim=[]
    # 코사인 유사도가 높은순으로 정렬
    k_i_j = sorted(c_s[i].items(), key=operator.itemgetter(1), reverse=True)

    for j in k_i_j:
      # 코사인 유사도
      sum_sim.append(j[1])
      # 코사인 유사도와 매출발생된 상품간의 예측계산
      sum_sim_rating.append( j[1]*sale_dict[addr][j[0]] )

    if len(sum_sim)>0:
      # 매출 발생하지 않은 상품에 대한 예측 매출수량
      result[i] = sum(sum_sim_rating)/sum(sum_sim)

  return sorted(result.items(), key=operator.itemgetter(1), reverse=True)

In [133]:
# 아이템 기반 필터링
recommendation_1000 = item_based_filtering(s_dict, '대구광역시 서구')
pd.DataFrame(np.array(recommendation_1000),columns=["상품명","예상판매수량"])


■■■ 상품별 유사도 현황 ■■■
예상 상품 :  생가득현미취나물솥밥(2인분)420G(풀무원)  매출발생 상품 :  ('매콤마늘퐁닭200G(롯데푸드)', 1.0)  유사도 :  ('목우촌프라임리얼동그랑땡300G*2(목우촌)', 1.0)
예상 상품 :  대림선핫도그800G(사조)  매출발생 상품 :  ('순살마늘퐁닭200G(롯데푸드)', 1.0)  유사도 :  ('매콤마늘퐁닭200G(롯데푸드)', 0.9999999999999998)


,상품명,예상판매수량
0,맛있는불닭철판볶음밥230G(오뚜기),6.320754716981132
1,쉐푸드한입에쏙들어가는김말이야채잡채400G(롯데푸드),6.175438596491228
2,동그랑땡500G(동원에프앤비),6.0476190476190474
3,야채바삭꼬마김말이(롯데푸드),5.690476190476191
4,오쉐프통살오징어바1KG(오뚜기),5.690476190476191
...,...,...
336,목우촌오븐구이떡갈비1KG(목우촌),4.519551823116704
337,명품동그랑땡1KG(동원에프앤비),4.454082463877296
338,새우)볶음밥(아워홈),4.427778984108566
339,초가삼간순희네빈대떡녹두빈대떡360G(롯데푸드),4.398208123187061
